# AgentKit SDK Tutorial 03: 流式模板launch和流式调用

本教程演示如何使用**流式模板**创建Agent项目，并进行**流式调用（Streaming Invocation）**。

在这个Notebook中，你将学会：

1. 使用 `sdk.init_project` 初始化一个**流式Agent项目**
2. 使用 `sdk.build` 构建流式Agent镜像
3. 使用 `sdk.deploy` 部署流式Agent
4. 使用 `sdk.invoke` 进行**流式调用**，并实时处理响应流
5. 了解流式响应的处理方式和最佳实践

**什么是流式调用？**
流式调用允许你实时接收Agent的响应内容，而不是等待整个响应完成。这对于长文本生成、对话系统等场景非常有用。

你可以直接从上到下依次运行每个代码单元。建议在**仓库根目录**或已安装好`agentkit`的Python环境中运行。

## 0. 环境准备

在开始本教程之前，请确保你已经**正确配置了凭证**。AgentKit需要两类凭证：

### 第一类：SDK与CLI的云服务访问凭证
用于创建和管理云资源（CR、CP、TOS、Runtime等）。推荐配置方式：

```bash
# 使用agentkit命令行工具全局配置（推荐）
agentkit config --global --set volcengine.access_key=YOUR_ACCESS_KEY
agentkit config --global --set volcengine.secret_key=YOUR_SECRET_KEY
agentkit config --global --set volcengine.region=cn-beijing
```

### 第二类：Agent运行时的环境变量
用于Agent访问大模型等服务。在`agentkit.yaml`的`runtime_envs`中配置(Cloud/Hybrid模式无需配置，Local模式需配置)：

```python
# 模型访问凭证（示例）
config.add_runtime_env("MODEL_AGENT_NAME", "your-model-name")
config.add_runtime_env("MODEL_AGENT_API_KEY", "your-model-api-key")
```

> **注意**：如果你还没有配置凭证，请先学习`00_configuration_and_credentials.ipynb`教程完成配置。

此外，请确保：
- 已安装Python 3.10+
- 已安装Jupyter/VSCode/PyCharm等可以运行Notebook的环境
- 已安装AgentKit SDK（通常在同一个代码仓库中开发，或通过pip安装）：

```bash
pip install agentkit-sdk-python  # 如果你是独立使用SDK
```

如果你是在AgentKit官方仓库中打开本Notebook，一般已经满足依赖，只需要确认当前工作目录在仓库根目录即可。

In [ ]:
# 导入本教程会用到的标准库和AgentKit SDK
import json
import asyncio
from pathlib import Path
from typing import AsyncIterator, Dict, Any

from agentkit.toolkit import sdk
from agentkit.toolkit.models import PreflightMode
from agentkit.toolkit.sdk import AgentKitClient, AgentConfig

print("AgentKit SDK已导入，版本环境就绪。")
print("本教程将演示流式Agent的创建和流式调用！")

## 1. 初始化流式Agent项目

我们使用`sdk.init_project`创建一个**流式Agent项目**：

- 项目目录：`tutorial_projects/t03-streaming-agent`
- 模板：`basic_stream`（支持流式响应的Agent模板）

`basic_stream`模板会生成：
- `agentkit.yaml`配置文件
- 支持流式响应的Agent代码
- SSE（Server-Sent Events）流式处理逻辑
- 必要的依赖文件

In [ ]:
# 约定项目路径
project_root = Path("tutorial_projects/t03-streaming-agent")
project_name = project_root.name

print(f"项目将创建在: {project_root.resolve()}")

# 调用SDK的init_project创建流式Agent工程
result = sdk.init_project(
    project_name=project_name,
    template="basic_stream",  # 关键：使用流式模板
    project_root=str(project_root),
    model_name="deepseek-v3-1-250821"
)

print("初始化是否成功:", result.success)
print("项目路径:", result.project_path)
print("生成的文件列表:")
for f in result.created_files:
    print(" -", f)

# 使用返回的project_path作为后续操作的项目根目录
project_root = Path(result.project_path)
config_file = project_root / "agentkit.yaml"
print("使用的配置文件:", config_file)

## 2. 查看流式Agent代码

让我们先查看生成的流式Agent代码，了解其工作原理：

In [ ]:
# 查看生成的Agent代码
agent_file = project_root / "t03-streaming-agent.py"
if agent_file.exists():
    print("=== 流式Agent代码 (agent.py) ===")
    with open(agent_file, 'r', encoding='utf-8') as f:
        content = f.read()
        # 显示关键部分
        lines = content.split('\n')
        for i, line in enumerate(lines[:50]):  # 显示前50行
            print(f"{i+1:3d}: {line}")
        if len(lines) > 50:
            print("    ... (省略剩余代码)")
else:
    print("未找到agent.py文件")

## 3. 配置流式Agent

让我们配置Agent的基本信息和运行时环境：

In [ ]:
# 加载配置
config = AgentConfig.load(project_root)

print("=== 原始配置 ===")
print(f"Agent名称: {config.agent_name}")
print(f"启动类型: {config.launch_type}")
print(f"描述: {config.description}")

# 修改配置以适应流式Agent
config.agent_name = "streaming-demo-agent"
config.description = "支持流式响应的演示Agent"
config.set_launch_type("cloud")  # 本地模式便于测试

# 保存配置
config.save()

print("\n=== 更新后的配置 ===")
print(f"Agent名称: {config.agent_name}")
print(f"启动类型: {config.launch_type}")
print(f"描述: {config.description}")
print(f"环境变量: {list(config.runtime_envs.keys())}")

## 4. 构建流式Agent镜像

使用`sdk.build`构建Agent镜像：

In [ ]:
print("开始构建流式Agent镜像...")

build_result = sdk.build(
    config_file=str(config_file),
    preflight_mode=PreflightMode.WARN,
)

print(f"构建结果: {'成功' if build_result.success else '失败'}")
if build_result.success:
    print(f"镜像信息: {build_result.image}")
else:
    print(f"错误信息: {build_result.error}")

## 5. 部署流式Agent

构建完成后，部署流式Agent到本地环境：

In [ ]:
print("开始部署流式Agent...")

deploy_result = sdk.deploy(
    config_file=str(config_file),
    preflight_mode=PreflightMode.WARN,
)

print(f"部署结果: {'成功' if deploy_result.success else '失败'}")
if deploy_result.success:
    print(f"服务端点: {deploy_result.endpoint_url}")
    endpoint_url = deploy_result.endpoint_url
else:
    print(f"错误信息: {deploy_result.error}")
    endpoint_url = None

## 6. 流式调用演示

现在让我们进行流式调用！我们将：

1. 创建一个测试请求
2. 使用流式模式调用Agent
3. 实时显示响应内容
4. 处理流式响应事件

In [ ]:
# 简洁版流式调用代码
async def simple_stream_invoke():
    """简洁的流式调用演示"""
    if not endpoint_url:
        print("请先成功部署Agent")
        return
    
    # 简单测试请求
    test_prompt = "你好"
    print(f"问题: {test_prompt}")
    print("流式响应: ", end="")
    
    # 流式调用
    result = sdk.invoke(
        payload={"prompt": test_prompt, "stream": True},
        config_file=str(config_file),
        headers={"user_id": "demo", "session_id": "test"}
    )
    
    if result.success and result.is_streaming:
        full_text = ""
        for chunk in result.stream():
            # 提取文本内容
            text = extract_text_from_chunk(chunk)
            if text:
                print(text, end='', flush=True)
                full_text += text
        print(f"\n\n总长度: {len(full_text)}字符")
    else:
        print("非流式响应:", result.response)

def extract_text_from_chunk(chunk):
    """从chunk中提取文本 - 简洁版本"""
    if isinstance(chunk, dict):
        # 处理字典格式
        if 'content' in chunk and chunk['content']:
            content = chunk['content']
            if isinstance(content, str):
                return content
            if 'parts' in content:
                return ''.join(part.get('text', '') for part in content['parts'])
    elif isinstance(chunk, str):
        try:
            # 尝试解析JSON字符串
            data = json.loads(chunk)
            return extract_text_from_chunk(data)
        except:
            return chunk
    return ""

await simple_stream_invoke()

## 7. 查看Agent状态

让我们检查流式Agent的运行状态：

In [ ]:
# 查看Agent状态
status_result = sdk.status(config_file=str(config_file))

print("=== Agent状态 ===")
print(f"状态查询结果: {'成功' if status_result.success else '失败'}")
if status_result.success:
    print(f"服务状态: {status_result.status}")
    print(f"端点URL: {status_result.endpoint_url}")
else:
    print(f"错误信息: {status_result.error}")

## 8. 流式调用最佳实践

基于本教程，我们总结一些流式调用的最佳实践：

### ✅ 推荐做法

1. **使用适当的流式模板**：选择`basic_stream`等支持流式的模板
2. **启用流式模式**：在payload中设置`"stream": True`
3. **正确处理SSE格式**：流式响应通常采用Server-Sent Events格式
4. **实时错误处理**：对流式响应进行异常捕获和处理
5. **性能监控**：统计响应时间、字符数等指标

### ⚠️ 注意事项

1. **检查is_streaming**：在调用`stream()`方法前，先检查`invoke_result.is_streaming`
2. **内存管理**：对于长文本，考虑分段处理避免内存溢出
3. **网络超时**：设置适当的超时时间处理长时间响应
4. **连接保持**：确保网络连接稳定，避免中断

## 9. 清理资源（可选）

测试完成后，你可以选择清理Runtime资源：

In [ ]:
# 清理本地部署的Agent（可选）
print("正在清理本地资源...")

# 使用destroy命令清理
destroy_result = sdk.destroy(config_file=str(config_file))

if destroy_result.success:
    print("✅ 本地资源清理完成")
else:
    print(f"❌ 清理失败: {destroy_result.error}")
    print("你可以手动停止Docker容器或删除相关资源")

print(f"\n项目文件保存在: {project_root}")
print("你可以保留项目用于后续实验，或手动删除整个目录")

## 🎉 教程完成！

恭喜！你已经成功完成了流式Agent的创建和流式调用教程。

### 📚 你学到了什么

- ✅ 如何使用流式模板创建Agent项目
- ✅ 如何构建和部署支持流式的Agent
- ✅ 如何进行流式调用并实时处理响应
- ✅ 如何处理SSE格式的流式数据
- ✅ 流式调用的最佳实践和注意事项

### 🔗 下一步

- 尝试其他模板和配置选项
- 集成记忆、知识库等服务
- 部署到本地环境
- 开发自定义的流式处理逻辑

如有问题，请参考[官方文档](https://volcengine.github.io/agentkit-sdk-python/)或提交Issue。